In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset


In [3]:
import pandas as pd

In [4]:
import random
random.seed(1)  # For consistency between the runs


In [5]:
class GenerateDataset():
  def __init__(self, dataset,length,is_train=True,min_length=4):
    self.dataset = load_dataset(dataset) # Can take a lot of time depending upon dataset
    if is_train:
      self.dataset=self.dataset["train"]
    self.length=length
    self.min_length=min_length
    self.dataset=self.get_dataset()
    self.__dataset=self.preprocess(self.dataset)  # For debugging purposes only
    self.dataset=self.gen_incomplete(self.__dataset)

  @property
  def complete_dataset(self):
    return self.__dataset

  def get_dataset(self,param=None):
    if param==None:
      premlim_li=random.choices(self.dataset,k=self.length)
    else:
      premlim_li=random.choices(self.dataset,k=param)
    return premlim_li

  def redundant_character(self,sentence,check=[" ","'","''"]):
    arr=sentence["text"].strip().split()
    count=len(arr)
    for i in arr:
      if i in check:
        count-=1
    return count

  def preprocess(self,li):
    buffer=[]
    count=0
    max_iter=100
    while True:
      max_iter-=1
      if max_iter==0:
        if len(buffer)<self.length:
          temp=self.get_dataset(self.length-len(buffer))
          buffer.extend([i["text"] for i in temp])
        break
      for i in li:
        if self.redundant_character(i)<=self.min_length:
          count+=1
        else:
          buffer.append(i["text"])
      if count==0 or len(buffer)==self.length:
        break
      else:
        li=self.get_dataset(count)
        count=0
    return buffer

  def gen_incomplete(self,pre_li):
    buffer=[]
    for i in pre_li:
      arr=i.strip().split()
      buffer.append(" ".join(arr[:(len(arr)//2)]))

    return buffer



  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    return self.dataset[idx]

  def toCSV(self):
    df=pd.DataFrame([i for i in self.dataset],columns=["x_i"])
    df.to_csv("dataset.csv")


In [7]:
data=GenerateDataset("bookcorpus",100)

Generating train split:   0%|          | 0/74004228 [00:00<?, ? examples/s]

In [8]:
data.toCSV()

In [13]:
data[45]

'a policy edgard hated and mocked at every opportunity ,'

In [15]:
# Load the dataset
df = pd.read_csv('dataset.csv')

# Replace all instances of "`" with an empty string in the entire DataFrame or a specific column
df_cleaned = df.applymap(lambda x: x.replace("`", "") if isinstance(x, str) else x)

# If you want to do this for a specific column, let's say 'text_column'
# df['text_column'] = df['text_column'].str.replace("``", "")

# Save the cleaned dataset
df_cleaned.to_csv('cleaned_dataset.csv', index=False)

# Display the cleaned data
df_cleaned.head()

<ipython-input-15-f82c29f2526c>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df.applymap(lambda x: x.replace("`", "") if isinstance(x, str) else x)


,Unnamed: 0,x_i
0,0,he picks up
1,1,or what
2,2,"when he finally entered her with a slow , draw..."
3,3,you did say that
4,4,the end ****** thank you for reading and suppo...
